In [ ]:
%pip install pandas

import pandas as pd

In [ ]:
movies = pd.read_csv('./data/TMDB_all_movies.csv')

movies = movies[['id', 'title', 'overview', 'genres', 'release_date']]
horror_movies = movies.loc[(movies['genres'] == 'Horror')]

horror_movies['tags'] = horror_movies['overview']+movies['genres']
horror_movies = horror_movies.drop(columns=['overview', 'genres'])
horror_movies.dropna()

In [ ]:
horror_movies.count()
# horror_movies.columns

In [ ]:
%pip install scikit-learn nltk

In [ ]:
# Text cleaning
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')

def clean_text(text):
    # Check if text is not a string
    if not isinstance(text, str):
        return ""

    # Convert to lowercase
    text = text.lower()

    # Remove punctuation but keep digits
    text = re.sub(r'[^\w\s\d]', '', text)

    # Tokenize
    words = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    # Join the words back together
    text = ' '.join(words)
    return text

In [ ]:
horror_movies['tags_clean'] = horror_movies['tags'].apply(clean_text)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(horror_movies, test_size=0.2, random_state=42)

cv = CountVectorizer(max_features=25000, stop_words='english')

In [ ]:
vector = cv.fit_transform(horror_movies['tags_clean'].values.astype('U')).toarray()

In [ ]:
vector.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity(vector)

In [ ]:
distance = sorted(list(enumerate(similarity[0])), reverse=True, key=lambda vector:vector[1])
for i in distance[0:5]:
  print(f"{horror_movies.iloc[i[0]].title} ({horror_movies.iloc[i[0]].release_date.split('-')[0]})")

In [ ]:
def recommend(movies):
  try:
    index = horror_movies[horror_movies['title'] == movies].index[0]
    distance = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda vector:vector[1])
    for i in distance[1:6]:
      print(f"{horror_movies.iloc[i[0]].title} ({horror_movies.iloc[i[0]].release_date.split('-')[0]})")
  except:
    print("Doesn't exist")

In [ ]:
recommend('Poltergeist')